# SHAP

In [1]:
import shap
import pandas as pd
import matplotlib.pyplot as plt

c:\ME\KAIM\week8\fraud-detection\week8\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the best model (Random Forest)
import joblib
rf_model = joblib.load('../models/fraud_rf_pipeline.joblib')

In [3]:
# import the dataset
x_train = pd.read_csv('../data/processed/x_train_final.csv')

In [4]:
# Select a small sample from X_test for faster computation
X_sample = x_train.sample(3, random_state=42)

In [5]:
# Extract the classifier from the pipeline
rf_clf = rf_model.named_steps["classifier"]  # just the trained RandomForest

In [6]:
print(f"Model: {rf_clf}")

Model: RandomForestClassifier(class_weight='balanced', random_state=42)


In [7]:
# Use TreeExplainer (recommended for tree-based models like RF)
# explainer = shap.TreeExplainer(rf_clf, x_train, model_output="raw", feature_perturbation="tree_path_dependent")
explainer = shap.TreeExplainer(rf_clf)

In [8]:
# Compute SHAP values
shap_values = explainer.shap_values(X_sample, check_additivity=False)

In [ ]:
# Plot SHAP summary
shap.summary_plot(shap_values[1], X_sample, feature_names=X_sample.columns)

---

In [ ]:
# Summary Plot
plt.figure()
shap.summary_plot(shap_values[1], X_test_transformed, feature_names=feature_names, show=False)
plt.title("SHAP Summary Plot - Fraud Class")
plt.tight_layout()
plt.savefig("../figures/shap_summary_plot.png")
plt.close()

# Force Plot for a single instance (first fraud case)
fraud_idx = y_test[X_test.index][X_test_sample.index].index[y_test[X_test_sample.index] == 1][0]
fraud_sample = X_test_transformed[X_test_sample.index.get_loc(fraud_idx)]
plt.figure()
shap.force_plot(explainer.expected_value[1], shap_values[1][X_test_sample.index.get_loc(fraud_idx)], fraud_sample, 
                feature_names=feature_names, matplotlib=True, show=False)
plt.title("SHAP Force Plot - Fraud Case")
plt.tight_layout()
plt.savefig("../figures/shap_force_plot.png")
plt.close()

# Save pipeline
joblib.dump(pipeline, "../models/fraud_rf_pipeline.joblib")

# Save feature names for FastAPI input validation
feature_names_input = X_train.columns.tolist()
joblib.dump(feature_names_input, "../models/feature_names.joblib")

# Generate SHAP report
with open("../reports/shap_report.txt", "w") as f:
    f.write("SHAP Analysis Report for Fraud Detection\n")
    f.write("=" * 40 + "\n\n")
    f.write("1. Model Performance\n")
    f.write(f"F1 Score: {f1_score(y_test, y_pred):.3f}\n")
    f.write(f"AUC-PR: {average_precision_score(y_test, y_proba):.3f}\n")
    f.write("Classification Report:\n")
    f.write(classification_report(y_test, y_pred) + "\n")
    f.write("\n2. Global Feature Importance (SHAP Summary Plot)\n")
    f.write("The SHAP Summary Plot (saved as ../figures/shap_summary_plot.png) shows the impact of each feature on fraud predictions:\n")
    f.write("- Features are ranked by their mean absolute SHAP values, indicating their overall importance.\n")
    f.write("- Red indicates higher feature values, blue indicates lower values.\n")
    f.write("- Positive SHAP values push predictions toward fraud (class=1), negative toward non-fraud (class=0).\n")
    f.write("\n3. Local Feature Importance (SHAP Force Plot)\n")
    f.write(f"The SHAP Force Plot (saved as ../figures/shap_force_plot.png) explains a single fraud case (index {fraud_idx}):\n")
    f.write("- Features with positive SHAP values increase the likelihood of fraud.\n")
    f.write("- Features with negative SHAP values decrease the likelihood.\n")
    f.write("\n4. Key Drivers of Fraud (Based on SHAP Values)\n")
    shap_importance = pd.DataFrame({
        'feature': feature_names,
        'mean_abs_shap': np.abs(shap_values[1]).mean(axis=0)
    }).sort_values('mean_abs_shap', ascending=False)
    f.write("Top 5 features driving fraud predictions:\n")
    for i, row in shap_importance.head(5).iterrows():
        f.write(f"- {row['feature']}: Mean |SHAP| = {row['mean_abs_shap']:.3f}\n")
    f.write("\nSee ../figures/shap_summary_plot.png and ../figures/shap_force_plot.png for visualizations.")